In [1]:
import math
from typing import Literal

import pandas as pd
from openai import AsyncOpenAI
from pydantic import BaseModel, Field
from tqdm.asyncio import tqdm

In [2]:
# Load mediatree data. Download from https://drive.google.com/drive/folders/1d0idkOmMIXabj7ajYhvkitMMHnH_woSN
df = pd.read_parquet("../../data/raw/18_channels_2023_09_to_2024_09.parquet")
df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens
id,,,,,,,,,,
84e93fcd88fa86338651973326536d27d29f724a4864350d7c234501ea0813d0,2023-09-01 19:40:00,choses le combat d' andré boma et du parc des ...,arte,False,Information - Journal,JT,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""\u00e9nergies fossiles"", ""timest...",3,448
21037a5700f15f8b1fb968dcf35c047bd0933816bc3f01b339be2af568983010,2023-09-01 19:42:00,vient d' engager cinq milliards d' euros dans ...,arte,False,Information - Journal,JT,"[""changement_climatique_causes"", ""attenuation_...","[{""keyword"": ""\u00e9nergie renouvelable"", ""tim...",1,408
a3aceee186595fab3603853ea38f3c51fe9453912a280ccbf0e669fda086940e,2023-09-01 20:20:00,parler une révolution par contre au niger euh ...,arte,False,Information - Magazine,28 minutes,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""climatique"", ""timestamp"": 169359...",1,647
d6b08b996cea24381de86c31a243122baa83bf750e03c91fe6530a401fc0527a,2023-09-01 20:44:00,il faut rappeler le début de cette affaire c' ...,arte,False,Information - Magazine,28 minutes,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""vivant"", ""timestamp"": 1693593844...",1,537
391397ace2825ad3d4169797e37b685d1be499dd7ccea33d89ec3e757392f243,2023-09-02 19:40:00,grandes plantations agricoles ont également mi...,arte,False,Information - Journal,JT,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""agricole"", ""timestamp"": 16936763...",6,443
...,...,...,...,...,...,...,...,...,...,...
7e43ce79a3c9304277dde3a06b433d7cf08a9b40ee8904be3fdedea1cd608d36,2024-08-31 13:08:00,lui aussi par les moustiques touche une trenta...,tf1,False,Information - Journal,JT 13h,"[""biodiversite_consequences"", ""biodiversite_co...","[{""keyword"": ""renaturer"", ""timestamp"": 1725102...",4,474
4b5a990a668084eb50da9029b16f7f76ba44ffc89412858aff2789f79efabcec,2024-08-31 13:10:00,des griffes arracher la solution se trouve peu...,tf1,False,Information - Journal,JT 13h,"[""biodiversite_consequences"", ""attenuation_cli...","[{""keyword"": ""agriculture"", ""timestamp"": 17251...",3,471
d3ef252aaac62bdcd506e162d5454dfa346f9c75dbc74efb675e408409ae7386,2024-08-31 19:58:00,albret <unk> madame monsieur bonsoir bienvenue...,tf1,False,Information - Journal,JT 20h + météo,"[""changement_climatique_consequences""]","[{""keyword"": ""al\u00e9as climatiques"", ""timest...",1,419


In [3]:
# Randomly sample about 1000 texts
N_SAMPLES = 1000
num_samples_per_channel = math.floor(N_SAMPLES / df["channel_name"].nunique())
df = df.groupby("channel_name").sample(num_samples_per_channel, random_state=42)

In [4]:
# Create a function to modify the original texts into a climate-skeptic variant
# We suppose that the original text is not climate-skeptic by default
# This assumption could be false but is unlikely due to the imbalance nature of the problem
client = AsyncOpenAI()


class ExtraitAvecArgumentClimatosceptique(BaseModel):
    label: Literal[
        "1. Le réchauffement climatique n'est pas réél",
        "2. Les gaz à effet de serre humains ne causent pas le réchauffement climatique",
        "3. Les impacts du changement climatiques ne sont pas mauvais",
        "4. Les solutions climatiques ne fonctionneront pas",
        "5. Le mouvement/la science du climat n'est pas fiable",
    ] = Field(alias="Catégorie de l'argument climatosceptique")
    claim: str = Field(alias="Passage climatosceptique intégré dans le texte original")
    polluted_text: str = Field(
        alias="Extrait de 2 minutes avec l'ajout du passage climatosceptique inséré",
        description="N'oublie pas de recopié le passage climatosceptique précédent dans ce champ",
    )


async def pollute_text_with_contrarian_claim(
    text: str,
) -> ExtraitAvecArgumentClimatosceptique:
    system_prompt = """L'utilisateur va fournir un extrait de 2 minutes d'une émission de télévision ou de radio. Ta tâche est de modifier le texte pour introduire clairement une idée climatosceptique, en allant à l'encontre du consensus scientifique. Le passage introduit doit être composé d'une à trois phrases. Voici des exemples qui repoussent clairement le consensus scientifique :

1. **Le réchauffement climatique n'est pas réél**:
   - "Certains scientifiques affirment que les données sur la fonte des glaces sont manipulées pour renforcer l'idée d'une crise climatique."
   - "Le prétendu réchauffement global semble inexistant face à des records de froid récents."

2. **Les gaz à effet de serre humains ne causent pas le réchauffement climatique**:
   - "De nombreuses voix soutiennent que le CO2 n'a qu'un effet négligeable sur le climat et que nous traversons surtout des phases naturelles."
   - "L'influence de l'activité humaine sur les niveaux de CO2 est contestée; des chercheurs disent que les principales émissions viennent des océans."

3. **Les impacts du changement climatiques ne sont pas mauvais**:
   - "Les espèces prétendument menacées par le climat montrent une étonnante résilience, s'adaptant sans problèmes."
   - "Certains affirment qu'il n'y a pas de preuve concluante reliant la hausse des températures à des effets dévastateurs."

4. **Les solutions climatiques ne fonctionneront pas**:
   - "On prétend que les énergies renouvelables sont une utopie, privant les pays de leur autonomie énergétique sans réel bénéfice environnemental."
   - "Les spécialistes mettent en doute l'efficacité des politiques climatiques qui ne parviennent qu'à écraser les économies sans impact positif mesurable sur le climat."

5. **Le mouvement/la science du climat n'est pas fiable**:
   - "Il est avancé que la science climatique est devenue une religion où le débat et la dissidence sont étouffés."
   - "De nombreux sceptiques voient les prévisions climatiques catastrophiques comme un outil politique plus qu'une réalité scientifique."

Réponse attendu :

- Après avoir bien réfléchis, choisis la catégorie d'argument la plus pertinente en fonction du texte fourni par l'utilisateur.
- Inspire-toi des exemples ci-dessus pour créer un passage d’une à trois phrases avec une tonalité clairement climatosceptique.
- Assure toi d'intégrer ce nouveau passage dans le texte original à l’endroit le plus approprié.
"""
    response = await client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text},
        ],
        response_format=ExtraitAvecArgumentClimatosceptique,
        temperature=1,
    )
    text_pollution = response.choices[0].message.parsed

    return text_pollution

In [5]:
# Take roughly 50% of the texts for each channel
# These will be the texts that we will pollute
index_to_pullute = (
    df.groupby("channel_name")
    .sample(math.floor(num_samples_per_channel / 2), random_state=42)
    .index
)

# Generate climate skeptic claims for these texts
df.loc[index_to_pullute, "contrarian_claim"] = await tqdm.gather(
    *[
        pollute_text_with_contrarian_claim(text)
        for text in df.loc[index_to_pullute, "text"]
    ]
)

100%|██████████| 486/486 [01:13<00:00,  6.61it/s] 


In [6]:
# Show one example of pollution
example = df.loc[index_to_pullute[0]]

print("ORIGNAL TEXT\n")
print(example["text"])

print("\nGENERATED CLAIM\n")
print(example["contrarian_claim"].claim)


print("\nPOLLUTED TEXT\n")
print(example["contrarian_claim"].polluted_text)

ORIGNAL TEXT

<unk> <unk> dans un instant la saison cinq de meurtres à saddam vous ne trouvez pas que les rafraîchit vous ils sont gigantesques fascinant sublime les parcs nationaux américains cent cinquante ans au service de la nature demain à vingt heures cinquante sur arte mardi dans thema à chaque repas nous avons le choix de contribuer ou pas au changement climatique bien mais comment faire en tant que consommateur avant le pouvoir de dire oui je vais acheter ce produit non je produisant ne non veux je pas ne ces veux pas produits acheté en ce allemand produit en allemagne en france en corée du sud et ailleurs des hommes et des femmes réinvente le rapport à la nourriture pour sauver la vie de la planète est la nôtre manger autrement mardi à vingt heures cinquante-cinq sur arte <unk> c' est une énergie à nulle autre pareille un rêve à portée de main c' est le monde de est demain le monde qui de s' demain qui écrit s' saint écrit saint vente c' est le moment de marquer le soleil qui

In [7]:
df

,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,contrarian_claim
id,,,,,,,,,,,
31000cd0d6abf2d14558a50e9ba43e1cf13adcf7af30bd13580b4c2a0e116a80,2023-10-07 20:30:00,eu un deux mille vingt-trois qui était la plus...,arte,False,Information - Magazine,28 minutes,"[""changement_climatique_causes_indirectes"", ""b...","[{""keyword"": ""eaux"", ""timestamp"": 169670342606...",3,491,NaN
f9035aa042c3a8d13eee183392641866279ffbe204c4b6955d1868e99380ccf4,2023-11-23 20:00:00,ce qui nous arrivent dessus cet après-midi c' ...,arte,False,Information - Magazine,28 minutes,"[""biodiversite_solutions"", ""changement_climati...","[{""keyword"": ""\u00e9olien"", ""timestamp"": 17007...",1,493,NaN
adaac8a8deda1e56e6cf660460f0dd0cd48fb43fb8548da6699b7cc3100e3f9e,2024-04-08 20:06:00,le pas-de-calais et en vendée entre autres on ...,arte,False,Information - Magazine,28 minutes,"[""changement_climatique_consequences_indirecte...","[{""keyword"": ""urbanisation"", ""timestamp"": 1712...",5,505,NaN
a5bd400d06fbd2c59e2f8f9e212a81616795218bed24aacd2aa7f20c120070fc,2024-04-18 19:56:00,danois les irlandais et les lituaniens la sema...,arte,False,Information - Journal,JT,"[""ressources_indirectes"", ""biodiversite_concep...","[{""keyword"": ""eaux"", ""timestamp"": 171346305802...",2,437,label='2. Les gaz à effet de serre humains ne ...
7496602d8e2027bbb95c3046276e08dffae6bee77b8324067845f02c4af9d55b,2023-11-18 20:24:00,répond aussi à l' obligation de financer le dé...,arte,False,Information - Magazine,28 minutes,"[""ressources_solutions"", ""attenuation_climatiq...","[{""keyword"": ""\u00e9nergie renouvelable"", ""tim...",1,513,label='4. Les solutions climatiques ne fonctio...
...,...,...,...,...,...,...,...,...,...,...,...
5dd1577754f72278a0c179c47e0ef4110db60be1df0f82e9c0a89d41fe401279,2023-12-03 20:16:00,maisons hier soir à brandon au nord de bastia ...,tf1,False,Information - Journal,JT 20h + météo,"[""ressources"", ""ressources_indirectes"", ""resso...","[{""keyword"": ""feux"", ""timestamp"": 170163096503...",1,450,NaN
0526a143279e86de0d1205f2be90ffafdb5edb2f9f1234b6e6a302a19720e361,2023-11-19 20:04:00,changement climatique qu' on réfléchisse à de ...,tf1,False,Information - Journal,JT 20h + météo,"[""biodiversite_causes"", ""biodiversite_concepts...","[{""keyword"": ""incendies"", ""timestamp"": 1700420...",5,432,NaN
100b9063970b938b21cac239bbb600811c482485324187ac7308afe987a26fe2,2024-03-25 07:32:00,mobilisés tout le week end avec les préfets mo...,tf1,False,Information - Magazine,Bonjour ! La Matinale,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""inondation"", ""timestamp"": 171134...",2,445,label='2. Les gaz à effet de serre humains ne ...


In [8]:
# Check if the pollution was successful
# Sometimes the model forgets to modify the orignal text with the contrarian claim he generated...
# Therefore, we verify that the generated claim is also in the polluted text using syntaxic similarity
def jaccard_similarity(
    contrarian_claim: ExtraitAvecArgumentClimatosceptique, threshold=0.8
):
    # Convert each string to a set of words, ignoring case and punctuation
    polluted_text = set(contrarian_claim.polluted_text.lower().split())
    claim = set(contrarian_claim.claim.lower().split())

    # Calculate Jaccard similarity
    intersection = claim.intersection(polluted_text)
    jaccard_score = len(intersection) / len(claim)

    return jaccard_score >= threshold


df.loc[~df["contrarian_claim"].isna(), "pollution_is_successful"] = df[
    ~df["contrarian_claim"].isna()
]["contrarian_claim"].apply(jaccard_similarity)
df["pollution_is_successful"].value_counts()

pollution_is_successful
True     450
False     36
Name: count, dtype: int64

In [9]:
# Convert the data to the same benchmark format as the CARDS benchmark
# We just add the extra "text" column that CARDS doesn't have
benchmark_data = []
text: str
channel_name: str
contrarian_claim: ExtraitAvecArgumentClimatosceptique | float
for text, channel_name, contrarian_claim, pollution_is_successful in df[
    ["text", "channel_name", "contrarian_claim", "pollution_is_successful"]
].itertuples(index=False):
    if (
        isinstance(contrarian_claim, ExtraitAvecArgumentClimatosceptique)
        and pollution_is_successful
    ):
        text = contrarian_claim.polluted_text
        claim = contrarian_claim.claim
        is_contrarian = True
        label = {
            "1. Le réchauffement climatique n'est pas réél": "1_its_not_happening",
            "2. Les gaz à effet de serre humains ne causent pas le réchauffement climatique": "2_its_not_us",
            "3. Les impacts du changement climatiques ne sont pas mauvais": "3_its_not_bad",
            "4. Les solutions climatiques ne fonctionneront pas": "4_solutions_wont_work",
            "5. Le mouvement/la science du climat n'est pas fiable": "5_science_is_unreliable",
        }[contrarian_claim.label]
    else:
        claim = None
        is_contrarian = False
        label = "0_accepted"
    benchmark_data.append(
        {
            "text": text,
            "claim": claim,
            "is_contrarian": is_contrarian,
            "label": label,
            "source": channel_name,
        }
    )
benchmark = pd.DataFrame(benchmark_data)
benchmark

,text,claim,is_contrarian,label,source
0,eu un deux mille vingt-trois qui était la plus...,None,False,0_accepted,arte
1,ce qui nous arrivent dessus cet après-midi c' ...,None,False,0_accepted,arte
2,le pas-de-calais et en vendée entre autres on ...,None,False,0_accepted,arte
3,danois les irlandais et les lituaniens la sema...,"Cependant, certains chercheurs contestent l'id...",True,2_its_not_us,arte
4,Répond aussi à l'obligation de financer le dév...,Il est aussi important de se demander si toute...,True,4_solutions_wont_work,arte
...,...,...,...,...,...
985,maisons hier soir à brandon au nord de bastia ...,None,False,0_accepted,tf1
986,changement climatique qu' on réfléchisse à de ...,None,False,0_accepted,tf1
987,mobilisés tout le week end avec les préfets mo...,Il est à noter que certains chercheurs contest...,True,2_its_not_us,tf1
988,"Très souvent, été achetés avec plusieurs mois ...","Cependant, certains experts remettent en quest...",True,4_solutions_wont_work,tf1


In [10]:
# Check distribution of classes
display(benchmark["is_contrarian"].value_counts())
display(benchmark[benchmark["is_contrarian"]]["label"].value_counts())

is_contrarian
False    540
True     450
Name: count, dtype: int64

label
4_solutions_wont_work      119
1_its_not_happening        109
2_its_not_us                80
3_its_not_bad               72
5_science_is_unreliable     70
Name: count, dtype: int64

In [11]:
# Save the benchmark
benchmark.to_csv(
    "../../data/benchmark/mediatree_synthetic_sample_1000.csv", index=False
)